# Fabric Item Management

## Imports & Constants

In [1]:
import requests
import time

from uuid import uuid4
from typing import Optional, Generic, TypeVar, Any, Type, Self, Iterator, Callable
from azure.identity import DefaultAzureCredential
from dataclasses import dataclass, asdict, field


StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 216, Finished, Available, Finished)

In [3]:
WORKSPACE_ID = "b1ccaa7d-dbba-4158-b98e-790aa7205600"
TOKEN = notebookutils.credentials.getToken("https://api.fabric.microsoft.com")
HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}
WORKSPACE_URL = f"https://api.fabric.microsoft.com/v1/workspaces/{WORKSPACE_ID}"

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 217, Finished, Available, Finished)

## Utils

In [4]:
class FabricAPIWorkspaceClient:
    def __init__(self, client):
        self._client = client
        self._base_url = f"{client.base_url}/workspaces"

    def get(self, workspace_id: str, item_path: str) -> requests.Response:
        url = self._url(workspace_id, item_path)
        resp: requests.Response = requests.get(url, headers=self._client.headers)
        return resp

    def post(self, workspace_id: str, item_path: str, payload: dict) -> requests.Response:
        url = self._url(workspace_id, item_path)
        resp: requests.Response = requests.post(
            url,
            headers=self._client.headers,
            json=payload
        )
        return resp

    def patch(self, workspace_id: str, item_path: str, payload: dict) -> requests.Response:
        url = self._url(workspace_id, item_path)
        resp: requests.Response = requests.patch(
            url,
            headers=self._client.headers,
            json=payload
        )
        return resp

    def put(self, workspace_id: str, item_path: str, payload: dict) -> requests.Response:
        url = self._url(workspace_id, item_path)
        resp: requests.Response = requests.put(
            url,
            headers=self._client.headers,
            json=payload
        )
        return resp

    def delete(self, workspace_id: str, item_path: str) -> requests.Response:
        url = self._url(workspace_id, item_path)
        resp: requests.Response = requests.delete(url, headers=self._client.headers)
        return resp

    def _url(self, workspace_id: str, item_path: str) -> str:
        item_path = self._prep_item_path(item_path)
        url = f"{self._base_url}/{workspace_id}{item_path}"
        return url

    def _prep_item_path(self, item_path: str) -> str:
        prep_item_path = item_path if item_path.startswith("/") else f"/{item_path}"
        return prep_item_path


class FabricAPIClient:
    def __init__(
        self,
        auth_type: str = None,
        version: str = "v1"
    ):
        self._base_url = f"https://api.fabric.microsoft.com/{version}"
        self.refresh_headers()

        self._workspaces = FabricAPIWorkspaceClient(self)

    @property
    def base_url(self) -> str:
        return self._base_url

    @property
    def workspaces(self) -> FabricAPIWorkspaceClient:
        return self._workspaces

    @property
    def headers(self) -> dict:
        return self._headers

    def refresh_headers(self) -> None:
        self._headers = {
            "Authorization": f"Bearer {self._get_token()}",
            "Content-Type": "application/json"
        }

    def get(self, path: str) -> requests.Response:
        url = self._url(path)
        resp: requests.Response = requests.get(url, headers=self.headers)
        return resp

    def post(self, path: str, payload: dict) -> requests.Response:
        url = self._url(path)
        resp: requests.Response = requests.post(
            url,
            headers=self.headers,
            json=payload
        )
        return resp

    def patch(self, path: str, payload: dict) -> requests.Response:
        url = self._url(path)
        resp: requests.Response = requests.patch(
            url,
            headers=self.headers,
            json=payload
        )
        return resp

    def put(self, path: str, payload: dict) -> requests.Response:
        url = self._url(path)
        resp: requests.Response = requests.put(
            url,
            headers=self.headers,
            json=payload
        )
        return resp

    def delete(self, path: str) -> requests.Response:
        url = self._url(path)
        resp: requests.Response = requests.delete(url, headers=self.headers)
        return resp

    def _url(self, path: str) -> str:
        path = self._prep_path(path)
        url = f"{self._base_url}{path}"
        return url

    def _prep_path(self, path: str) -> str:
        prep_item_path = path if path.startswith("/") else f"/{item_path}"
        return prep_item_path

    def _get_token(self) -> str:
        token = notebookutils.credentials.getToken("https://api.fabric.microsoft.com") # TODO
        return token

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 218, Finished, Available, Finished)

In [5]:
client = FabricAPIClient()

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 219, Finished, Available, Finished)

## BaseItem Implementation

In [6]:
TItemAPIData = TypeVar("TItemAPIData")


@dataclass
class FabricItem(Generic[TItemAPIData]):
    fields: dict[str, Any]
    apiData: Optional[TItemAPIData] = None

    def __init__(self, apiData: Optional[TItemAPIData] = None, **fields: dict):
        self.fields = fields
        self.apiData = apiData


class BaseItem(Generic[TItemAPIData]):
    def __init__(
            self,
            create_type_fn: Callable,
            base_item_url: str,
            workspace_id: str,
            item: FabricItem[TItemAPIData]
    ):
        self._create_type_fn = create_type_fn
        self._base_item_url = base_item_url
        self._workspace_id = workspace_id
        self._item: FabricItem[TItemAPIData] = item

    @property
    def item(self) -> FabricItem[TItemAPIData]:
        return self._item

    @staticmethod
    def get_by_id(
            create_type_fn: Callable,
            workspace_id: str,
            base_item_url: str,
            id: str
    ) -> Any:
        item_path = f"{base_item_url}/{id}"
        resp = client.workspaces.get(workspace_id, item_path)
        resp.raise_for_status()
        item = resp.json()
        print("ITEM BY ID:", item)
        return create_type_fn(item)

    @staticmethod
    def get_by_name(
            create_type_fn: Callable,
            workspace_id: str,
            base_item_url: str,
            name: str
    ) -> Any:
        item_path = base_item_url
        resp = client.workspaces.get(workspace_id, item_path)
        resp.raise_for_status()
        for item in resp.json()["value"]:
            if item["displayName"] == name:
                return create_type_fn(item)
        raise LookupError(f"{cls.__name__} '{name}' not found")

    @staticmethod
    def list(workspace_id: str, base_item_url: str) -> Iterator[Any]:
        item_path = base_item_url
        resp = client.workspaces.get(workspace_id, item_path)
        resp.raise_for_status()
        for item in resp.json()["value"]:
            yield item

    def fetch(self) -> Self:
        if self._item.apiData is None:
            self._item.apiData = BaseItem.get_by_name(
                create_type_fn=self._create_type_fn,
                workspace_id=self._workspace_id,
                base_item_url=self._base_item_url,
                name=self._item.fields["displayName"]
            ).item.apiData
            return self
        
        self._item.apiData = BaseItem.get_by_id(
            create_type_fn=self._create_type_fn,
            workspace_id=self._workspace_id,
            base_item_url=self._base_item_url,
            id=self._item.apiData.id
        ).item.apiData
        return self

    def exists(self) -> bool:
        try:
            return self.fetch()._item.apiData is not None
        except Exception as e: # TODO: nur bei not found; nicht bei allen exp
            return False
        
    def create(self) -> None:
        item_path = self._base_item_url
        payload = self._item.fields
        resp = client.workspaces.post(
            workspace_id=self._workspace_id,
            item_path=item_path,
            payload=payload
        )
        resp.raise_for_status()
        item = resp.json()
        if resp.status_code == 202 and item is None:
            item = self._wait_after_202(resp)

        print("item:", item)
        self._item.apiData = self._create_type_fn(item).item.apiData
        self.fetch()

    def create_if_not_exists(self) -> None:
        if self.exists():
            return
        self.create()

    def update(self) -> None:
        if self._item.apiData is None:
            self.fetch()

        item_path = f"{self._base_item_url}/{self._item.apiData.id}"
        payload = self._item.fields
        resp = client.workspaces.patch(
            workspace_id=self._workspace_id,
            item_path=item_path,
            payload=payload
        )
        resp.raise_for_status()
        item = resp.json()
        self._item.apiData = self._create_type_fn(item).item.apiData
        self.fetch()

    def delete(self) -> None:
        if self._item.apiData is None:
            self.fetch()

        item_path = f"{self._base_item_url}/{self._item.apiData.id}"
        resp = client.workspaces.delete(
            workspace_id=self._workspace_id,
            item_path=item_path
        )
        resp.raise_for_status()

    def _wait_after_202(self, resp: requests.Response) -> requests.Response:
        op_id = resp.headers["x-ms-operation-id"]
        op_location = resp.headers["Location"]
        retry = int(resp.headers.get("Retry-After", 10))
        print(f"Status=202, wait for success, retry={retry} OP={op_id}")
        warehouse = None
        while True:
            time.sleep(retry)
            s = requests.get(op_location, headers=client.headers)
            # Wenn fertig, zeigt Location auf .../result
            if s.json().get("status") == "Succeeded":
                res = requests.get(s.headers["Location"], headers=client.headers)
                res.raise_for_status()
                warehouse = res.json()
                break
            retry = int(s.headers.get("Retry-After", retry))
            print(f"Wait for more {retry}s")

        return warehouse


    def __str__(self) -> str:
        item_str = str(self._item)
        item_str_total = (
            f"{self.__class__.__name__}("
            f"workspaceId='{self._workspace_id}', "
            f"item={item_str}"
        )
        return item_str_total

    def __repr__(self) -> str:
        return str(self)

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 220, Finished, Available, Finished)

## Notebook

## VariableLibrary

In [ ]:
client.workspaces.get(WORKSPACE_ID, "/VariableLibraries").json()

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 221, Finished, Available, Finished)

{'value': [{'id': '312682a8-935c-4fe4-9dca-be458217deb2',
   'type': 'VariableLibrary',
   'displayName': 'Environment Variables',
   'description': '',
   'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600',
   'properties': {'activeValueSetName': 'Default value set'}},
  {'id': '55a1955c-1f99-46a0-9b12-c482189a8976',
   'type': 'VariableLibrary',
   'displayName': 'Environment Var-1',
   'description': 'No description',
   'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600',
   'properties': {'activeValueSetName': 'Default value set'}},
  {'id': 'aab4328b-8c26-4deb-9acb-ebfc659cadbc',
   'type': 'VariableLibrary',
   'displayName': 'Environment Var-4-u',
   'description': 'No description',
   'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600',
   'properties': {'activeValueSetName': 'Default value set'}},
  {'id': '685ee539-3c63-4065-a088-2f609c0fa7c7',
   'type': 'VariableLibrary',
   'displayName': 'Environment Var-10',
   'description': 'No description',
   'workspaceId': '

In [ ]:
var0 = client.workspaces.get(WORKSPACE_ID, "/VariableLibraries").json()["value"][0]


resp = client.workspaces.post(WORKSPACE_ID, f"/VariableLibraries/{var0['id']}/getDefinition", payload={})
time.sleep(1)
resp = requests.get(resp.headers["Location"], headers=client.headers)
resp = requests.get(resp.headers["Location"], headers=client.headers)

definition = resp.json()["definition"]
definition

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 222, Finished, Available, Finished)

{'parts': [{'path': 'variables.json',
   'payload': 'ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRlbS92YXJpYWJsZUxpYnJhcnkvZGVmaW5pdGlvbi92YXJpYWJsZXMvMS4wLjAvc2NoZW1hLmpzb24iLA0KICAidmFyaWFibGVzIjogWw0KICAgIHsNCiAgICAgICJuYW1lIjogIlZhcmlhYmxlMSIsDQogICAgICAibm90ZSI6ICIiLA0KICAgICAgInR5cGUiOiAiU3RyaW5nIiwNCiAgICAgICJ2YWx1ZSI6ICJibHViLWRlZmF1bHQiDQogICAgfSwNCiAgICB7DQogICAgICAibmFtZSI6ICJWYXJpYWJsZTIiLA0KICAgICAgIm5vdGUiOiAiIiwNCiAgICAgICJ0eXBlIjogIlN0cmluZyIsDQogICAgICAidmFsdWUiOiAidmFyMiINCiAgICB9DQogIF0NCn0=',
   'payloadType': 'InlineBase64'},
  {'path': 'settings.json',
   'payload': 'ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRlbS92YXJpYWJsZUxpYnJhcnkvZGVmaW5pdGlvbi9zZXR0aW5ncy8xLjAuMC9zY2hlbWEuanNvbiIsDQogICJ2YWx1ZVNldHNPcmRlciI6IFsNCiAgICAiVEVTVCIsDQogICAgIlBST0QiDQogIF0NCn0=',
   'payloadType': 'InlineBase64'},
  {'path': 'valueSets/TEST.json',
   'payload': 'ew0KICAiJHNjaGVtYSI

In [ ]:
parts = definition["parts"]
parts

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 223, Finished, Available, Finished)

[{'path': 'variables.json',
  'payload': 'ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRlbS92YXJpYWJsZUxpYnJhcnkvZGVmaW5pdGlvbi92YXJpYWJsZXMvMS4wLjAvc2NoZW1hLmpzb24iLA0KICAidmFyaWFibGVzIjogWw0KICAgIHsNCiAgICAgICJuYW1lIjogIlZhcmlhYmxlMSIsDQogICAgICAibm90ZSI6ICIiLA0KICAgICAgInR5cGUiOiAiU3RyaW5nIiwNCiAgICAgICJ2YWx1ZSI6ICJibHViLWRlZmF1bHQiDQogICAgfSwNCiAgICB7DQogICAgICAibmFtZSI6ICJWYXJpYWJsZTIiLA0KICAgICAgIm5vdGUiOiAiIiwNCiAgICAgICJ0eXBlIjogIlN0cmluZyIsDQogICAgICAidmFsdWUiOiAidmFyMiINCiAgICB9DQogIF0NCn0=',
  'payloadType': 'InlineBase64'},
 {'path': 'settings.json',
  'payload': 'ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2RldmVsb3Blci5taWNyb3NvZnQuY29tL2pzb24tc2NoZW1hcy9mYWJyaWMvaXRlbS92YXJpYWJsZUxpYnJhcnkvZGVmaW5pdGlvbi9zZXR0aW5ncy8xLjAuMC9zY2hlbWEuanNvbiIsDQogICJ2YWx1ZVNldHNPcmRlciI6IFsNCiAgICAiVEVTVCIsDQogICAgIlBST0QiDQogIF0NCn0=',
  'payloadType': 'InlineBase64'},
 {'path': 'valueSets/TEST.json',
  'payload': 'ew0KICAiJHNjaGVtYSI6ICJodHRwczovL2Rl

In [ ]:
@dataclass
class VariableLibraryProperties:
    activeValueSetName: Optional[str] = None


@dataclass
class VariableLibraryAPIData:
    id: str
    workspaceId: str
    displayName: str
    description: Optional[str]
    type: Optional[str]
    properties: VariableLibraryProperties = field(default_factory=VariableLibraryProperties)

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 224, Finished, Available, Finished)

In [ ]:
@dataclass
class VariableLibraryVariable:
    name: str
    note: str
    type: str
    value: Any


def _b64(obj: dict) -> str:
    json_bytes = json.dumps(obj, ensure_ascii=True).encode("utf-8")
    obj = base64.b64encode(json_bytes).decode("ascii")
    return obj


class VariableLibraryDefinition:
    def __init__(self, variable_lib_name: str, value_sets_ordered: list[str], *variables: list[VariableLibraryVariable]) -> None:
        self._var_lib_name = variable_lib_name
        self._value_sets = value_sets_ordered
        self._variables = variables or []

    @property
    def name(self) -> str:
        return self._var_lib_name

    def to_definition(self) -> list[dict]:
        variables = [asdict(var) for var in self._variables]
        variables_data = {
            "$schema": f"https://developer.microsoft.com/json-schemas/fabric/item/variableLibrary/definition/variables/1.0.0/schema.json",
            "variables": variables
        }
        settings_data = {
            "$schema": f"https://developer.microsoft.com/json-schemas/fabric/item/variableLibrary/definition/settings/1.0.0/schema.json",
            "valueSetsOrder": self._value_sets
        }
        platform_data = {
            "$schema": f"https://developer.microsoft.com/json-schemas/fabric/gitIntegration/platformProperties/2.0.0/schema.json",
            "metadata": {
                "type": "VariableLibrary",
                "displayName": self._var_lib_name
            },
            "config": {
                "version": "2.0",
                "logicalId": "00000000-0000-0000-0000-000000000000"
            }
        }

        variable_sets_data_b64 = [
            {
                "path": f"valueSets/{value_set}.json",
                "payload": _b64({
                    "$schema": "https://developer.microsoft.com/json-schemas/fabric/item/variableLibrary/definition/valueSet/1.0.0/schema.json",
                    "name": value_set,
                    "variableOverrides": [
                        {"name": var["name"], "value": var["value"]}
                        for var in variables
                    ]
                }),
                "payloadType": "InlineBase64"
            }
            for value_set in self._value_sets
        ]

        variables_data_b64 = _b64(variables_data)
        settings_data_b64 = _b64(settings_data)
        platform_data_b64 = _b64(platform_data)

        parts = [
            {
                "path": "variables.json",
                "payload": variables_data_b64,
                "payloadType": "InlineBase64"
            },
            {
                "path": "settings.json",
                "payload": settings_data_b64,
                "payloadType": "InlineBase64"
            }
        ] + variable_sets_data_b64 + [
            {
                "path": ".platform",
                "payload": platform_data_b64,
                "payloadType": "InlineBase64"
            }
        ]

        return {
            "parts": parts
        }

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 225, Finished, Available, Finished)

In [ ]:
class VariableLibrary(BaseItem[VariableLibraryAPIData]):
    """
    REF: https://learn.microsoft.com/en-us/rest/api/fabric/variablelibrary/items
    """
    def __init__(
        self,
        workspace_id: str,
        name: str,
        description: str = None,
        folder_id: str = None,
        definition: VariableLibraryDefinition = None,
        api_data: VariableLibraryAPIData = None
    ):
        definition = definition.to_definition() if isinstance(definition, VariableLibraryDefinition) else None
        description = description or "New VariableLibrary"
        item = FabricItem[VariableLibraryAPIData](
            displayName=name,
            description=description,
            folderId=folder_id,
            definition=definition,
            apiData=api_data
        )
        super().__init__(
            create_type_fn=VariableLibrary.from_json,
            base_item_url="/VariableLibraries",
            workspace_id=workspace_id,
            item=item
        )

    @staticmethod
    def from_json(item: dict) -> "VariableLibraryAPIData":
        kwargs = item.copy()
        if not "properties" in item.keys():
            item["properties"] = {}
        kwargs["properties"] = VariableLibraryProperties(**item["properties"])
        api_data = VariableLibraryAPIData(**kwargs)
        return VariableLibrary(
            workspace_id=api_data.workspaceId,
            name=api_data.displayName,
            description=api_data.description,
            api_data=api_data
        )

    @staticmethod
    def get_by_name(workspace_id: str, name: str) -> VariableLibraryAPIData:
        return BaseItem.get_by_name(
            create_item_type_fn=VariableLibrary.from_json,
            workspace_id=workspace_id,
            base_item_url="/VariableLibraries",
            name=name
        )

    @staticmethod
    def get_by_id(workspace_id: str, id: str) -> VariableLibraryAPIData:
        return BaseItem.get_by_id(
            create_item_type_fn=VariableLibrary.from_json,
            workspace_id=workspace_id,
            base_item_url="/VariableLibraries",
            id=id
        )

    @staticmethod
    def list(workspace_id: str) -> list[VariableLibraryAPIData]:
        return [
            VariableLibrary.from_json(item)
            for item in BaseItem.list(
                workspace_id=workspace_id,
                base_item_url="/VariableLibraries"
            )
        ]


varlib = VariableLibrary(
    WORKSPACE_ID,
    name="VariableLibe"
)
print(varlib)

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 226, Finished, Available, Finished)

VariableLibrary(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'VariableLibe', 'description': 'New VariableLibrary', 'folderId': None, 'definition': None}, apiData=None)


In [ ]:
name = f"VL_{uuid4().hex[:10]}"

definition = VariableLibraryDefinition(
    name,
    ["TEST", "PROD"],
    VariableLibraryVariable(
        name="Variable1",
        note="",
        type="String",
        value="blub-default"
    ),
    VariableLibraryVariable(
        name="Variable2",
        note="",
        type="String",
        value="var2"
    )
)

varlib_new = VariableLibrary(
    workspace_id=WORKSPACE_ID,
    name=name,
    definition=definition
)

print(name)
varlib_new.create()
# varlib_new.delete()

StatementMeta(, b61d4a64-b53d-4a93-9497-e7126714f892, 228, Finished, Available, Finished)

Status=202, wait for success, retry=20 OP=6bac7ef7-6b74-40da-ace9-d778541a3337


WAIT JSON: {'status': 'Succeeded', 'createdTimeUtc': '2025-08-16T18:39:07.4905283', 'lastUpdatedTimeUtc': '2025-08-16T18:39:10.3967931', 'percentComplete': 100, 'error': None}
item: {'id': '4ba20307-eda8-440d-9963-d79e96f2e804', 'type': 'VariableLibrary', 'displayName': 'VL_90df91f885', 'description': 'New VariableLibrary', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
ITEM BY ID: {'id': '4ba20307-eda8-440d-9963-d79e96f2e804', 'type': 'VariableLibrary', 'displayName': 'VL_90df91f885', 'description': 'New VariableLibrary', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600', 'properties': {'activeValueSetName': 'Default value set'}}


## Warehouse

In [7]:
@dataclass
class WarehouseProperties:
    connectionInfo: Optional[str] = None
    connectionString: Optional[str] = None
    createdDate: Optional[str] = None
    creationMode: Optional[str] = None
    sourceRestorePoint: Optional[str] = None
    collationType: Optional[str] = None
    lastUpdatedTime: Optional[str] = None


@dataclass
class WarehouseAPIData:
    id: str
    workspaceId: str
    displayName: str
    description: Optional[str]
    type: str
    properties: WarehouseProperties = field(default_factory=WarehouseProperties)


StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 87, Finished, Available, Finished)

In [8]:
class Warehouse(BaseItem[WarehouseAPIData]):
    """
    REF: https://learn.microsoft.com/en-us/rest/api/fabric/warehouse/items
    """
    def __init__(
        self,
        workspace_id: str,
        name: str,
        description: str = None,
        folder_id: str = None,
        collation_type: str = "Latin1_General_100_BIN2_UTF8",
        api_data: WarehouseAPIData = None
    ):
        description = description or "New Warehouse"
        item = FabricItem[WarehouseAPIData](
            displayName=name,
            description=description,
            folderId=folder_id,
            creationPayload={
                "collationType": collation_type
            },
            apiData=api_data
        )
        super().__init__(
            create_type_fn=Warehouse.from_json,
            base_item_url="/warehouses",
            workspace_id=workspace_id,
            item=item
        )

    @staticmethod
    def from_json(item: dict) -> "WarehouseAPIData":
        kwargs = item.copy()
        if not "properties" in item.keys():
            item["properties"] = {}
        kwargs["properties"] = WarehouseProperties(**item["properties"])
        api_data = WarehouseAPIData(**kwargs)
        return Warehouse(
            workspace_id=api_data.workspaceId,
            name=api_data.displayName,
            description=api_data.description,
            api_data=api_data
        )

    @staticmethod
    def get_by_name(workspace_id: str, name: str) -> WarehouseAPIData:
        return BaseItem.get_by_name(
            create_item_type_fn=Warehouse.from_json,
            workspace_id=workspace_id,
            base_item_url="/warehouse",
            name=name
        )

    @staticmethod
    def get_by_id(workspace_id: str, id: str) -> WarehouseAPIData:
        return BaseItem.get_by_id(
            create_item_type_fn=Warehouse.from_json,
            workspace_id=workspace_id,
            base_item_url="/warehouse",
            id=id
        )

    @staticmethod
    def list(workspace_id: str) -> list[WarehouseAPIData]:
        return [
            Warehouse.from_json(item)
            for item in BaseItem.list(
                workspace_id=workspace_id,
                base_item_url="/warehouse"
            )
        ]


wh = Warehouse(WORKSPACE_ID, name="PlaygroundWarehouse")
print(wh)

StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 88, Finished, Available, Finished)

Warehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'PlaygroundWarehouse', 'description': 'New Warehouse', 'folderId': None, 'creationPayload': {'collationType': 'Latin1_General_100_BIN2_UTF8'}}, apiData=None)


In [9]:
import time

name = f"WH_{uuid4().hex[:10]}"
wh_new = Warehouse(WORKSPACE_ID, name=name)
wh_new.create()
print(f"CREATED: {name}")
wh_new

StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 89, Finished, Available, Finished)

Status=202, wait for success, retry=20 OP=d4b8637b-798e-4aaa-ae5d-161d1315e4b6


WAIT JSON: {'status': 'Succeeded', 'createdTimeUtc': '2025-08-16T15:16:24.2435824', 'lastUpdatedTimeUtc': '2025-08-16T15:16:36.5264967', 'percentComplete': 100, 'error': None}
item: {'id': '9b8c5121-e634-49de-be68-e77e4dc32ed7', 'type': 'Warehouse', 'displayName': 'WH_219a53ae10', 'description': 'New Warehouse', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600'}
ITEM BY ID: {'id': '9b8c5121-e634-49de-be68-e77e4dc32ed7', 'type': 'Warehouse', 'displayName': 'WH_219a53ae10', 'description': 'New Warehouse', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600', 'properties': {'connectionInfo': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'connectionString': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'createdDate': '2025-08-16T15:16:23.9935813', 'lastUpdatedTime': '2025-08-16T15:16:36.4015046', 'collationType': 'Latin1_General_100_BIN2_UTF8', 'creationMode': 'New', 'sourceRestorePoint': None}

Warehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'WH_219a53ae10', 'description': 'New Warehouse', 'folderId': None, 'creationPayload': {'collationType': 'Latin1_General_100_BIN2_UTF8'}}, apiData=WarehouseAPIData(id='9b8c5121-e634-49de-be68-e77e4dc32ed7', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='WH_219a53ae10', description='New Warehouse', type='Warehouse', properties=WarehouseProperties(connectionInfo='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', connectionString='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', createdDate='2025-08-16T15:16:23.9935813', creationMode='New', sourceRestorePoint=None, collationType='Latin1_General_100_BIN2_UTF8', lastUpdatedTime='2025-08-16T15:16:36.4015046')))

In [ ]:
wh_new.fetch()

StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 90, Finished, Available, Finished)

ITEM BY ID: {'id': '9b8c5121-e634-49de-be68-e77e4dc32ed7', 'type': 'Warehouse', 'displayName': 'WH_219a53ae10', 'description': 'New Warehouse', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600', 'properties': {'connectionInfo': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'connectionString': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'createdDate': '2025-08-16T15:16:23.9935813', 'lastUpdatedTime': '2025-08-16T15:16:36.4015046', 'collationType': 'Latin1_General_100_BIN2_UTF8', 'creationMode': 'New', 'sourceRestorePoint': None}}


Warehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'WH_219a53ae10', 'description': 'New Warehouse', 'folderId': None, 'creationPayload': {'collationType': 'Latin1_General_100_BIN2_UTF8'}}, apiData=WarehouseAPIData(id='9b8c5121-e634-49de-be68-e77e4dc32ed7', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='WH_219a53ae10', description='New Warehouse', type='Warehouse', properties=WarehouseProperties(connectionInfo='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', connectionString='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', createdDate='2025-08-16T15:16:23.9935813', creationMode='New', sourceRestorePoint=None, collationType='Latin1_General_100_BIN2_UTF8', lastUpdatedTime='2025-08-16T15:16:36.4015046')))

In [ ]:
wh_new.item.fields["description"] = "Updated description"
wh_new.update()

wh_new

StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 91, Finished, Available, Finished)

ITEM BY ID: {'id': '9b8c5121-e634-49de-be68-e77e4dc32ed7', 'type': 'Warehouse', 'displayName': 'WH_219a53ae10', 'description': 'Updated description', 'workspaceId': 'b1ccaa7d-dbba-4158-b98e-790aa7205600', 'properties': {'connectionInfo': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'connectionString': 'a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', 'createdDate': '2025-08-16T15:16:23.9935813', 'lastUpdatedTime': '2025-08-16T15:18:20.2625873', 'collationType': 'Latin1_General_100_BIN2_UTF8', 'creationMode': 'New', 'sourceRestorePoint': None}}


Warehouse(workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', item=FabricItem(fields={'displayName': 'WH_219a53ae10', 'description': 'Updated description', 'folderId': None, 'creationPayload': {'collationType': 'Latin1_General_100_BIN2_UTF8'}}, apiData=WarehouseAPIData(id='9b8c5121-e634-49de-be68-e77e4dc32ed7', workspaceId='b1ccaa7d-dbba-4158-b98e-790aa7205600', displayName='WH_219a53ae10', description='Updated description', type='Warehouse', properties=WarehouseProperties(connectionInfo='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', connectionString='a6ri6ubdhgfubhe2p537uuy4hy-pwvmzmn23nmedomopefkoicwaa.datawarehouse.fabric.microsoft.com', createdDate='2025-08-16T15:16:23.9935813', creationMode='New', sourceRestorePoint=None, collationType='Latin1_General_100_BIN2_UTF8', lastUpdatedTime='2025-08-16T15:18:20.2625873')))

In [ ]:
wh_new.delete()

StatementMeta(, 03650e40-c81c-4b52-bd4d-2ba4af4632ea, 92, Finished, Available, Finished)

## Folder

In [55]:
"""
curl -X POST \
  "https://api.fabric.microsoft.com/v1/workspaces/${WORKSPACE_ID}/folders" \
  -H "Authorization: Bearer $ACCESS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{
        "displayName": "Data",
        "parentFolderId": null
      }'
"""

resp = client.workspaces.post(WORKSPACE_ID, "/folders", {
    "displayName": "newFolder",
    "parentFolderId": None
})

print(resp.status_code)
print(resp.text)

StatementMeta(, 94472719-572d-4b40-b18f-ed0020b12ce7, 57, Finished, Available, Finished)

201
{"id":"04fe8565-87d9-47c5-bc0a-d41957ad660e","displayName":"newFolder","workspaceId":"b1ccaa7d-dbba-4158-b98e-790aa7205600"}


In [56]:
resp = client.workspaces.get(WORKSPACE_ID, "/folders")
print(resp.text)

StatementMeta(, 94472719-572d-4b40-b18f-ed0020b12ce7, 58, Finished, Available, Finished)

{"value":[{"id":"04fe8565-87d9-47c5-bc0a-d41957ad660e","displayName":"newFolder","workspaceId":"b1ccaa7d-dbba-4158-b98e-790aa7205600"}]}


## Lakehouse

In [ ]:
@dataclass
class LakehouseSqlEndpointProperties:
    id: Optional[str] = None
    connectionString: Optional[str] = None
    provisioningStatus: Optional[str] = None


@dataclass
class LakehouseProperties:
    defaultSchema: Optional[str] = None
    oneLakeTablesPath: Optional[str] = None
    oneLakeFilesPath: Optional[str] = None
    LakehouseSqlEndpointProperties: LakehouseSqlEndpointProperties = field(default_factory=LakehouseSqlEndpointProperties)


@dataclass
class LakehouseAPIData:
    id: str
    workspaceId: str
    displayName: str
    description: Optional[str]
    type: str
    properties: LakehouseProperties = field(default_factory=LakehouseProperties)


In [ ]:
class Lakehouse(BaseItem[LakehouseAPIData]):
    """
    REF: https://learn.microsoft.com/en-us/rest/api/fabric/lakehouse/items
    """
    def __init__(
        self,
        workspace_id: str,
        name: str,
        description: str = None,
        folder_id: str = None,
        enable_schemas: bool = True,
        api_data: LakehouseAPIData = None
    ):
        description = description or "New Lakehouse"
        item = FabricItem[LakehouseAPIData](
            displayName=name,
            description=description,
            folderId=folder_id,
            creationPayload={
                "enableSchemas": enable_schemas
            },
            apiData=api_data
        )
        super().__init__(
            create_type_fn=Lakehouse.from_json,
            base_item_url="/lakehouses",
            workspace_id=workspace_id,
            item=item
        )

    @staticmethod
    def from_json(item: dict) -> "LakehouseAPIData":
        kwargs = item.copy()
        if not "properties" in item.keys():
            item["properties"] = {
                "LakehouseSqlEndpointProperties": {}
            }
        kwargs["properties"] = LakehouseProperties(**item["properties"])
        kwargs["properties"].LakehouseSqlEndpointProperties = LakehouseSqlEndpointProperties(
            **item["properties"]["LakehouseSqlEndpointProperties"]
        )
        api_data = LakehouseAPIData(**kwargs)
        return Lakehouse(
            workspace_id=api_data.workspaceId,
            name=api_data.displayName,
            description=api_data.description,
            api_data=api_data
        )

    
    @staticmethod
    def get_by_name(workspace_id: str, name: str) -> LakehouseAPIData:
        return BaseItem.get_by_name(
            create_item_type_fn=Lakehouse.from_json,
            workspace_id=workspace_id,
            base_item_url="/lakehouses",
            name=name
        )

    @staticmethod
    def get_by_id(workspace_id: str, id: str) -> LakehouseAPIData:
        return BaseItem.get_by_id(
            create_item_type_fn=Lakehouse.from_json,
            workspace_id=workspace_id,
            base_item_url="/lakehouses",
            id=id
        )

    @staticmethod
    def list(workspace_id: str) -> list[LakehouseAPIData]:
        return [
            Lakehouse.from_json(item)
            for item in BaseItem.list(
                workspace_id=workspace_id,
                base_item_url="/lakehouses"
            )
        ]


lh = Lakehouse(WORKSPACE_ID, name="PlaygroundLakehouse")
print(lh)

In [ ]:
lh.fetch()

print(lh)

In [ ]:
lh.item.apiData.properties.LakehouseSqlEndpointProperties.provisioningStatus

In [ ]:
import time


lh_new = Lakehouse(WORKSPACE_ID, name=f"LH_{uuid4().hex[:10]}")
lh_new.create()
print("CREATED")

while lh_new.fetch().item.apiData.properties.LakehouseSqlEndpointProperties.provisioningStatus != "Success":
    state = lh_new.item.apiData.properties.LakehouseSqlEndpointProperties.provisioningStatus
    print("CURRENT STATE:", state)
    time.sleep(2.5)


state = lh_new.item.apiData.properties.LakehouseSqlEndpointProperties.provisioningStatus
print("CURRENT STATE:", state)
lh_new

In [ ]:
lh_new.fetch() # provisioningStatus InProgress -> noch nicht fertig!

In [ ]:
lh_new.item.fields["description"] = "Updated description"
lh_new.update()

lh_new

In [ ]:
lh_new.create_if_not_exists()

In [ ]:
lh_new.delete()

In [ ]:
lakehouses = Lakehouse.list(WORKSPACE_ID)

len(lakehouses)

## Warehouse

# TODOs

In [ ]:
# NUN die Logik in eine BaseClass auslagern
# und für Warehouses etc bauen
# hier mit Generic arbeiten


# from fabricengineer.api.fabric.items import Lakehouse, Warehouse, VariableLibrary, ...